Ce code effectue plusieurs étapes de prétraitement des données, y compris l'imputation des valeurs manquantes, l'encodage ordinal des variables catégorielles, l'encodage one-hot des variables catégorielles avec plus d'une catégorie, et l'application d'une transformation Smooth Ridit aux variables numériques. Ensuite, il entraîne deux modèles, une régression ElasticNet et un modèle LightGBM avec une perte de Poisson, sur les données prétraitées. Enfin, il sauvegarde le modèle et le mapping des catégories d'attaques pour une utilisation ultérieure.

In [3]:
import logging
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_poisson_deviance
from sklearn.linear_model import ElasticNet
import lightgbm as lgb
import pandas as pd
import numpy as np

# Configuration de la journalisation
logging.basicConfig(filename='model.log', level=logging.INFO)

def load_data(filepath):
    logging.info("Chargement des données...")
    data = pd.read_csv(filepath, encoding='ISO-8859-1')
    return data

def preprocess_data(data):
    logging.info("Prétraitement des données...")
    # Imputer les valeurs manquantes pour les variables numériques
    numeric_features = data.select_dtypes(include=['float64']).columns
    imputer = SimpleImputer(strategy='median')
    data[numeric_features] = imputer.fit_transform(data[numeric_features])

    # Appliquer la transformation Smooth Ridit aux caractéristiques numériques dans le DataFrame d'entraînement
    numeric_features = data.select_dtypes(include=[np.number]).columns
    data[numeric_features] = smooth_ridit_transform(data[numeric_features])

    # Ordinal encoding pour les variables catégorielles
    categorical_features = data.select_dtypes(include=['object']).columns
    ordinal_encoder = OrdinalEncoder()
    data[categorical_features] = ordinal_encoder.fit_transform(data[categorical_features])

    # Vérifier le nombre de catégories uniques dans chaque colonne catégorielle
    unique_counts = data[categorical_features].nunique()

    # Filtrer les colonnes qui ont plus d'une catégorie
    multi_cat_features = unique_counts[unique_counts > 1].index

    # Appliquer l'encodage One-Hot uniquement sur les colonnes avec plus d'une catégorie
    encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
    encoded_cat = encoder.fit_transform(data[multi_cat_features])
    encoded_df = pd.DataFrame(encoded_cat, columns=encoder.get_feature_names_out(multi_cat_features))

    # Supprimer les colonnes catégorielles originales
    data.drop(multi_cat_features, axis=1, inplace=True)

    # Concaténer les données encodées avec les données originales
    df_encoded = pd.concat([data, encoded_df], axis=1)

    return df_encoded

def train_model(df_encoded, target_column):
    logging.info("Entraînement du modèle...")
    # Régression ElasticNet pour les variables numériques et catégorielles combinées
    X = df_encoded.drop(target_column, axis=1)
    y = df_encoded[target_column]

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    imputer = SimpleImputer(strategy='median')
    X_train_imputed = imputer.fit_transform(X_train_scaled)
    X_val_imputed = imputer.transform(X_val_scaled)
    enet = ElasticNet(alpha=0.5, l1_ratio=0.7, max_iter=10000)
enet.fit(X_train_imputed, y_train)
enet_pred_train = enet.predict(X_train_imputed)
enet_pred_val = enet.predict(X_val_imputed)
mse = mean_squared_error(y_val, enet_pred_val)
print(f"Mean Squared Error: {mse}")

# LightGBM avec perte de Poisson
dtrain = lgb.Dataset(X_train, label=y_train)
dval = lgb.Dataset(X_val, label=y_val)
params = {
    'boosting_type': 'gbdt',
    'objective': 'poisson',
    'metric': 'poisson',
    'learning_rate': 0.11,
}

# Définir le callback early_stopping
early_stopping = lgb.callback.early_stopping(stopping_rounds=50, verbose=True)
model = lgb.train(params, dtrain, num_boost_round=1000, valid_sets=[dval], callbacks=[early_stopping])

y_pred = model.predict(X_val, num_iteration=model.best_iteration)
poisson_deviance = mean_poisson_deviance(y_val, y_pred)
print(f"Poisson Deviance: {poisson_deviance}")

# Sauvegarder le modèle et le mapping
joblib.dump(model, 'model.pkl')
joblib.dump(mapping, 'mapping.pkl')

# Charger le modèle et le mapping
model = joblib.load('model.pkl')
mapping = joblib.load('mapping.pkl')



NameError: name 'enet' is not defined

In [30]:
nombre_de_colonnes = len(train.columns)

# Afficher le nombre de colonnes
print(f"Le DataFrame a {nombre_de_colonnes} colonnes.")

Le DataFrame a 44.


In [31]:

# Appliquez votre transformation ici
print(f"Colonnes après la transformation : {train.columns}")

Colonnes avant la transformation : Index(['sport', 'dsport', 'dur', 'sbytes', 'dbytes', 'sttl', 'dttl', 'sloss',
       'dloss', 'sload', 'dload', 'spkts', 'dpkts', 'swin', 'dwin', 'stcpb',
       'dtcpb', 'smeansz', 'dmeansz', 'trans_depth', 'res_bdy_len', 'sjit',
       'djit', 'stime', 'ltime', 'sintpkt', 'dintpkt', 'tcprtt', 'synack',
       'ackdat', 'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd',
       'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm',
       'ct_src_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
       'attack_cat', 'label'],
      dtype='object')
Colonnes après la transformation : Index(['sport', 'dsport', 'dur', 'sbytes', 'dbytes', 'sttl', 'dttl', 'sloss',
       'dloss', 'sload', 'dload', 'spkts', 'dpkts', 'swin', 'dwin', 'stcpb',
       'dtcpb', 'smeansz', 'dmeansz', 'trans_depth', 'res_bdy_len', 'sjit',
       'djit', 'stime', 'ltime', 'sintpkt', 'dintpkt', 'tcprtt', 'synack',
       'ackdat', 'is_sm_ips_ports',

In [26]:
# Colonnes originales
colonnes_originales = train.columns

# Colonnes après la transformation
colonnes_transformees = df_encoded.columns

# Colonnes manquantes
colonnes_manquantes = set(colonnes_originales) - set(colonnes_transformees)

# Afficher les colonnes manquantes
print(f"Colonnes manquantes : {colonnes_manquantes}")

Colonnes manquantes : set()


In [20]:
original_columns = X_train.columns
new_data_columns = new_data.columns
missing_columns = set(original_columns) - set(new_data_columns)
extra_columns = set(new_data_columns) - set(original_columns)

In [27]:
# Prédire sur de nouvelles données
new_data = pd.read_csv('E:\M2i\RNCP\RNCP_CDC\DataBase\sample_train5.csv')

def preprocess_data(data, imputer, encoder, scaler, train_columns):
    # Ajouter des fonctionnalités manquantes
    missing_cols = set(train_columns) - set(data.columns)
    for c in missing_cols:
        data[c] = np.nan

    # Assurer que l'ordre des colonnes est le même que dans les données d'entraînement
    data = data[train_columns]

    # Imputer les valeurs manquantes
    numeric_features = data.select_dtypes(include=['float64']).columns
    data[numeric_features] = imputer.transform(data[numeric_features])

    # Appliquer la transformation Smooth Ridit
    numeric_features = data.select_dtypes(include=[np.number]).columns
    data[numeric_features] = smooth_ridit_transform(data[numeric_features])

    # Encoder en One-Hot
    categorical_features = data
    
# Encoder en One-Hot
encoded_cat_new_data = encoder.transform(new_data[categorical_features])
encoded_df_new_data = pd.DataFrame(encoded_cat_new_data, columns=encoder.get_feature_names_out(categorical_features))

# Supprimer les colonnes catégorielles originales dans les nouvelles données
new_data.drop(categorical_features, axis=1, inplace=True)

# Concaténer les nouvelles colonnes One-Hot avec les données originales
# Continuer le prétraitement des nouvelles données
new_data = pd.concat([new_data, encoded_df_new_data], axis=1)

# Prétraiter new_data
new_data_preprocessed = preprocess_data(new_data, imputer, ordinal_encoder, scaler, original_columns)

# Faire des prédictions sur new_data
new_data_predictions = model.predict(new_data_preprocessed)

# Convertir les prédictions en catégories d'attaques
new_data_attack_cats = pd.Series(new_data_predictions).map(mapping)

# Afficher les catégories d'attaques prédites
print(new_data_attack_cats)

C:\Users\steph\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_encoders.py:228: UserWarning: Found unknown categories in columns [0, 1, 2, 3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\steph\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


ValueError: X has 155 features, but SimpleImputer is expecting 187 features as input.

In [22]:
# Faire des prédictions sur new_data
new_data_predictions = model.predict(new_data_preprocessed)

# Convertir les prédictions en catégories d'attaques
new_data_attack_cats = pd.Series(new_data_predictions).map(mapping)

NameError: name 'new_data_preprocessed' is not defined